In [1]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg


import numpy as np
from scipy.fft import fft2, ifft2, fftfreq, fftshift

from scipy.spatial import Delaunay
from scipy.interpolate import griddata
from scipy.spatial.qhull import QhullError

from tkinter import messagebox


    




# Global variables to store data and file path
d = None
file_path = None




def browse_file():
    global d, file_path
    file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
    if file_path:
        d = pd.read_excel(file_path)

# Function to plot on a canvas
def plot_on_canvas(plot_func):
    if d is not None:
        plt.figure()
        plot_func()
        plt.show()

# Function to plot in a new window
def plot_in_new_window(plot_func, *args, **kwargs):
    if d is not None:
        new_window = tk.Toplevel(root)
        plot_func(*args, **kwargs, window=new_window)
        
# Function to plot in a new window
# def plot_in_new_window2(plot_func, *args, **kwargs):
#     if d is not None:
#         new_window = tk.Toplevel(root)
#         plot_func(*args, **kwargs, window=new_window,grid_resolution=str(grid_resolution[0]))


def theortical_anomaly(d, gabs_base,window):
    #Extracting Reading and time for drift correction
    reading = d.iloc[:,1]
    s = d.iloc[:,0]
    time = d.iloc[:,6]
    theta = d.iloc[:,3]

    # Coordinates of the first and last points
    x1, y1 = time.iloc[0], reading.iloc[0]
    x2, y2 = time.iloc[-1], reading.iloc[-1]

    # Calculate the distance from each point to the horizontal line
    dh= np.abs(reading - y1)

    # Calculate the distance from each point to the line connecting the first and last points
    dp= np.abs((y2 - y1) * time - (x2 - x1) * reading + x2 * y1 - y2 * x1) / np.sqrt((y2 - y1)**2 + (x2 - x1)**2)

    drift = np.abs(dh-dp)
    drift[0]=0
    relative_reading=[]
    
    drift_correction = reading - drift
    for i in range (len(drift_correction)):
        relative_reading.append(drift_correction[i]-drift_correction[0])
    
    abs_reading = []
    for i in relative_reading:
        abs_reading.append(i+float(gabs_base))
        
    #abs_reading = [x +  gabs_base for x in relative_reading]
    
    theortical_reading = []
    for i in range (len(theta)):
        theortical_reading.append(978032.7*(1+0.0053024*((np.sin(theta[i]*(np.pi/180)))**2)-0.0000058*(np.sin(2*theta[i]*(np.pi/180)))**2))
    anamoly = []
    for i in range (len(theortical_reading)):
        anamoly.append(abs_reading[i] - theortical_reading[i])
    
        
    fig, ax = plt.subplots()
    ax.plot(s, anamoly)
    ax.set_xlabel('Station Coordinates')
    ax.set_ylabel('Theortical Anomaly')
    ax.set_title('Theoretical Anomaly')
    canvas = FigureCanvasTkAgg(fig, master=window)
    canvas.draw()
    canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)


def faa(d,gabs_base,window):
    elevation = d.iloc[:,2]
    s = d.iloc[:,0]
    reading = d.iloc[:,1]
    time = d.iloc[:,6]
    theta = d.iloc[:,3]

    # Coordinates of the first and last points
    x1, y1 = time.iloc[0], reading.iloc[0]
    x2, y2 = time.iloc[-1], reading.iloc[-1]

    # Calculate the distance from each point to the horizontal line
    dh= np.abs(reading - y1)

    # Calculate the distance from each point to the line connecting the first and last points
    dp= np.abs((y2 - y1) * time - (x2 - x1) * reading + x2 * y1 - y2 * x1) / np.sqrt((y2 - y1)**2 + (x2 - x1)**2)

    drift = np.abs(dh-dp)
    drift[0]=0
    relative_reading=[]
    
    drift_correction = reading - drift
    for i in range (len(drift_correction)):
        relative_reading.append(drift_correction[i]-drift_correction[0])
    
    abs_reading = []
    for i in relative_reading:
        abs_reading.append(i+float(gabs_base))
        
        
    #abs_reading = [x +  gabs_base for x in relative_reading]
    
    theortical_reading = []
    for i in range (len(theta)):
        theortical_reading.append(978032.7*(1+0.0053024*((np.sin(theta[i]*(np.pi/180)))**2)-0.0000058*(np.sin(2*theta[i]*(np.pi/180)))**2))
    anamoly = []
    for i in range (len(theortical_reading)):
        anamoly.append(abs_reading[i] - theortical_reading[i])
    
    fac =[]
    for i in range (len(elevation)):
        fac.append(0.3086*elevation[i])
    
        
    fa_anomaly = []
    for i in range (len(fac)):
        fa_anomaly.append(anamoly[i]+fac[i])
        
        
        
    fig, ax = plt.subplots()
    ax.plot(s, fa_anomaly)
    ax.set_xlabel('Station Coordinates')
    ax.set_ylabel('Free-Air Anomaly')
    ax.set_title('Free-Air  Anomaly')
    canvas = FigureCanvasTkAgg(fig, master=window)
    canvas.draw()
    canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)
    
    
def ba(d,gabs_base,roh_g_cc,window):
    #Extracting Reading and time for drift correction
    reading = d.iloc[:,1]
    s = d.iloc[:,0]
    time = d.iloc[:,6]
    theta = d.iloc[:,3]

    # Coordinates of the first and last points
    x1, y1 = time.iloc[0], reading.iloc[0]
    x2, y2 = time.iloc[-1], reading.iloc[-1]

    # Calculate the distance from each point to the horizontal line
    dh= np.abs(reading - y1)

    # Calculate the distance from each point to the line connecting the first and last points
    dp= np.abs((y2 - y1) * time - (x2 - x1) * reading + x2 * y1 - y2 * x1) / np.sqrt((y2 - y1)**2 + (x2 - x1)**2)

    drift = np.abs(dh-dp)
    drift[0]=0
    relative_reading=[]
    
    
    drift_correction = reading - drift
    for i in range (len(drift_correction)):
        relative_reading.append(drift_correction[i]-drift_correction[0])
    
    abs_reading = []
    for i in relative_reading:
        abs_reading.append(i+float(gabs_base))
        
        
#     abs_reading = [x +  gabs_base for x in relative_reading]
    
    theortical_reading = []
    for i in range (len(theta)):
        theortical_reading.append(978032.7*(1+0.0053024*((np.sin(theta[i]*(np.pi/180)))**2)-0.0000058*(np.sin(2*theta[i]*(np.pi/180)))**2))
    anomaly = []
    for i in range (len(theortical_reading)):
        anomaly.append(abs_reading[i] - theortical_reading[i])
    
    elevation = d.iloc[:,2]
    fac =[]
    for i in range (len(elevation)):
        fac.append(0.3086*elevation[i])
        
    fa_anomaly = []
    for i in range (len(fac)):
        fa_anomaly.append(anomaly[i]+fac[i]) 
        
    #Calculation of bouger correction 
    bg_corr = []
    for i in range (len(elevation)):
        bg_corr.append(0.04192*float(roh_g_cc)*elevation[i])
    bg_anomaly = []
    for i in range (len(bg_corr)):
        bg_anomaly.append(fa_anomaly[i]-bg_corr[i])
        
        
    fig, ax = plt.subplots()
    ax.plot(s, bg_anomaly)
    ax.set_xlabel('Station Coordinates')
    ax.set_ylabel('Bouguer Anomaly')
    ax.set_title('Bouguer Anomaly')
    canvas = FigureCanvasTkAgg(fig, master=window)
    canvas.draw()
    canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

    
def upward_continuation(d,gabs_base,roh_g_cc,elevation_change,grid_resolution, window):
      #Extracting Reading and time for drift correction
    reading = d.iloc[:,1]
    s = d.iloc[:,0]
    time = d.iloc[:,6]
    theta = d.iloc[:,3]
    
    latitude = d.iloc[:,3]
    longitude = d.iloc[:,7] 

    # Coordinates of the first and last points
    x1, y1 = time.iloc[0], reading.iloc[0]
    x2, y2 = time.iloc[-1], reading.iloc[-1]

    # Calculate the distance from each point to the horizontal line
    dh= np.abs(reading - y1)

    # Calculate the distance from each point to the line connecting the first and last points
    dp= np.abs((y2 - y1) * time - (x2 - x1) * reading + x2 * y1 - y2 * x1) / np.sqrt((y2 - y1)**2 + (x2 - x1)**2)

    drift = np.abs(dh-dp)
    drift[0]=0
    relative_reading=[]
    
    
    drift_correction = reading - drift
    for i in range (len(drift_correction)):
        relative_reading.append(drift_correction[i]-drift_correction[0])
    
    abs_reading = []
    for i in relative_reading:
        abs_reading.append(i+float(gabs_base))
        
        
#     abs_reading = [x +  gabs_base for x in relative_reading]
    
    theortical_reading = []
    for i in range (len(theta)):
        theortical_reading.append(978032.7*(1+0.0053024*((np.sin(theta[i]*(np.pi/180)))**2)-0.0000058*(np.sin(2*theta[i]*(np.pi/180)))**2))
    anomaly = []
    for i in range (len(theortical_reading)):
        anomaly.append(abs_reading[i] - theortical_reading[i])
    
    elevation = d.iloc[:,2]
    fac =[]
    for i in range (len(elevation)):
        fac.append(0.3086*elevation[i])
        
    fa_anomaly = []
    for i in range (len(fac)):
        fa_anomaly.append(anomaly[i]+fac[i]) 
        
    #Calculation of bouger correction 
    bg_corr = []
    for i in range (len(elevation)):
        bg_corr.append(0.04192*float(roh_g_cc)*elevation[i])
    bg_anomaly = []
    for i in range (len(bg_corr)):
        bg_anomaly.append(fa_anomaly[i]-bg_corr[i])
        
        

    
    # Create a meshgrid of latitude and longitude
    lat_grid, lon_grid = np.meshgrid(np.linspace(min(latitude), max(latitude), grid_resolution[0]),
                                     np.linspace(min(longitude), max(longitude), grid_resolution[1]))

    try:
        # Interpolate gravity readings onto the grid using cubic interpolation
        interpolated_gravity = griddata((latitude, longitude), bg_anomaly, (lat_grid, lon_grid), method='linear')
    except QhullError:
        # If cubic interpolation fails, use nearest interpolation as a fallback
        interpolated_gravity = griddata((latitude, longitude), bg_anomaly, (lat_grid, lon_grid), method='nearest')

    
    # Number of rows and columns in the gravity data
    rows, cols = interpolated_gravity.shape

    # Calculate the spatial frequencies for rows and columns
    freq_rows = fftfreq(rows, d=1)
    freq_cols = fftfreq(cols, d=1)

    # Perform 2D Fourier Transform
    ft_gravity = fft2(interpolated_gravity)

    # Apply shift to move zero frequency component to the center
    ft_gravity_shifted = fftshift(ft_gravity)

    # Construct the filter in frequency domain to remove high frequencies
    freq_filter_rows = np.exp(-2j * np.pi * float(elevation_change) * freq_rows)
    freq_filter_cols = np.exp(-2j * np.pi * float(elevation_change) * freq_cols)
    freq_filter = np.outer(freq_filter_rows, freq_filter_cols)

    # Apply the filter in frequency domain
    ft_gravity_filtered = ft_gravity_shifted * freq_filter

    # Inverse Fourier Transform to get upward continued gravity data
    upward_continued_data = ifft2(fftshift(ft_gravity_filtered))

    # Take real part since the result may have small imaginary components due to numerical errors
    upward_continued_data = np.real(upward_continued_data)

    fig, ax = plt.subplots()
    ax.plot(s, upward_continued_data)
    ax.set_xlabel('Station Coordinates')
    ax.set_ylabel('Bouguer Anomaly')
    ax.set_title('Upward Continuation')
    canvas = FigureCanvasTkAgg(fig, master=window)
    canvas.draw()
    canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

    
    
def open_array_input_window():
    global array_input
    array_input = tk.Toplevel(root)
    array_input.title("Input Array")

    array_label = tk.Label(array_input, text="Enter the array elements separated by space:")
    array_label.pack()

    global array_entry
    array_entry = tk.Entry(array_input)
    array_entry.pack()

    submit_button = tk.Button(array_input, text="Submit", command=get_array_input)
    submit_button.pack()

# Function to get array input from the user
def get_array_input():
    global array_input
    global array_entry
    array_elements = array_entry.get().split()
    try:
        global input_array
        input_array = [int(elem) for elem in array_elements]
        array_input.destroy()
        messagebox.showinfo("Success", "Array input successful!")
    except ValueError:
        messagebox.showerror("Error", "Please enter valid numeric values!")
        
        
        
root=tk.Tk()
root.title("Anomaly Calculator")

browse_button = tk.Button(root, text="Browse File", command=lambda:browse_file())
browse_button.pack()

gabs_base_label = tk.Label(root, text="Enter g_abs_base:")
gabs_base_label.pack()
gabs_base = tk.Entry(root)
gabs_base.pack()

roh_g_label = tk.Label(root, text="Enter roh_g (g/cc):")
roh_g_label.pack()
roh_g_entry = tk.Entry(root)
roh_g_entry.pack()

# elevation_change_label = tk.Label(root, text ='Adjust Elevation for Upward Continuation')
# elevation_change_label.pack()
# elevation_change = tk.Entry(root)
# elevation_change.pack()

# grid_resolution = tk.Button(root, text="Input Grid Resolution", command=open_array_input_window)
# grid_resolution.pack()


anomaly1_button = tk.Button(root, text="Theortical Anomaly", command=lambda: plot_in_new_window(theortical_anomaly,d,gabs_base.get()))
anomaly1_button.pack()
anomaly2_button = tk.Button(root, text="Free Air Anomaly", command=lambda: plot_in_new_window(faa,d, gabs_base.get()))
anomaly2_button.pack()
anomaly3_button = tk.Button(root, text="Bouguer Anomaly ", command=lambda: plot_in_new_window(ba,d, gabs_base.get(),roh_g_entry.get()))
anomaly3_button.pack()

root.mainloop()

C:\Users\yriti\AppData\Local\Temp\ipykernel_9076\1970485070.py:14: DeprecationWarning: Please use `QhullError` from the `scipy.spatial` namespace, the `scipy.spatial.qhull` namespace is deprecated.
  from scipy.spatial.qhull import QhullError


In [5]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from scipy.fft import fft2, ifft2, fftfreq, fftshift
from scipy.spatial.qhull import QhullError
from scipy.interpolate import griddata
from tkinter import messagebox


# Global variables to store data and file path
d = None
file_path = None


def browse_file():
    global d, file_path
    file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
    if file_path:
        d = pd.read_excel(file_path)


# Function to plot on a canvas
def plot_on_canvas(plot_func):
    if d is not None:
        plt.figure()
        plot_func()
        plt.show()


# Function to plot in a new window
def plot_in_new_window(plot_func, *args, **kwargs):
    if d is not None:
        new_window = tk.Toplevel(root)
        plot_func(*args, **kwargs, window=new_window)


def theoretical_anomaly(d, gabs_base, window):
    # Extracting Reading and time for drift correction
    reading = d.iloc[:, 1]
    s = d.iloc[:, 0]
    time = d.iloc[:, 6]
    theta = d.iloc[:, 3]

    # Coordinates of the first and last points
    x1, y1 = time.iloc[0], reading.iloc[0]
    x2, y2 = time.iloc[-1], reading.iloc[-1]

    # Calculate the distance from each point to the horizontal line
    dh = np.abs(reading - y1)

    # Calculate the distance from each point to the line connecting the first and last points
    dp = np.abs((y2 - y1) * time - (x2 - x1) * reading + x2 * y1 - y2 * x1) / np.sqrt((y2 - y1) ** 2 + (x2 - x1) ** 2)

    drift = np.abs(dh - dp)
    drift[0] = 0
    relative_reading = []

    drift_correction = reading - drift
    for i in range(len(drift_correction)):
        relative_reading.append(drift_correction[i] - drift_correction[0])

    abs_reading = []
    for i in relative_reading:
        abs_reading.append(i + float(gabs_base))

    # Theoretical readings
    theoretical_reading = []
    for i in range(len(theta)):
        theoretical_reading.append(
            978032.7 * (1 + 0.0053024 * ((np.sin(theta[i] * (np.pi / 180))) ** 2) - 0.0000058 * (
                    np.sin(2 * theta[i] * (np.pi / 180))) ** 2))
    anomaly = [abs_reading[i] - theoretical_reading[i] for i in range(len(theoretical_reading))]

    fig, ax = plt.subplots()
    ax.plot(s, anomaly)
    ax.set_xlabel('Station Coordinates')
    ax.set_ylabel('Theoretical Anomaly')
    ax.set_title('Theoretical Anomaly')
    canvas = FigureCanvasTkAgg(fig, master=window)
    canvas.draw()
    canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)


def faa(d, gabs_base, window):
    elevation = d.iloc[:, 2]
    s = d.iloc[:, 0]
    reading = d.iloc[:, 1]
    time = d.iloc[:, 6]
    theta = d.iloc[:, 3]

    # Coordinates of the first and last points
    x1, y1 = time.iloc[0], reading.iloc[0]
    x2, y2 = time.iloc[-1], reading.iloc[-1]

    # Calculate the distance from each point to the horizontal line
    dh = np.abs(reading - y1)

    # Calculate the distance from each point to the line connecting the first and last points
    dp = np.abs((y2 - y1) * time - (x2 - x1) * reading + x2 * y1 - y2 * x1) / np.sqrt((y2 - y1) ** 2 + (x2 - x1) ** 2)

    drift = np.abs(dh - dp)
    drift[0] = 0
    relative_reading = []

    drift_correction = reading - drift
    for i in range(len(drift_correction)):
        relative_reading.append(drift_correction[i] - drift_correction[0])

    abs_reading = [x + float(gabs_base) for x in relative_reading]

    # Theoretical readings
    theoretical_reading = []
    for i in range(len(theta)):
        theoretical_reading.append(
            978032.7 * (1 + 0.0053024 * ((np.sin(theta[i] * (np.pi / 180))) ** 2) - 0.0000058 * (
                    np.sin(2 * theta[i] * (np.pi / 180))) ** 2))
    anomaly = [abs_reading[i] - theoretical_reading[i] for i in range(len(theoretical_reading))]

    # Free-air correction
    fac = [0.3086 * ele for ele in elevation]

    fa_anomaly = [anomaly[i] + fac[i] for i in range(len(fac))]

    fig, ax = plt.subplots()
    ax.plot(s, fa_anomaly)
    ax.set_xlabel('Station Coordinates')
    ax.set_ylabel('Free-Air Anomaly')
    ax.set_title('Free-Air Anomaly')
    canvas = FigureCanvasTkAgg(fig, master=window)
    canvas.draw()
    canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)


def ba(d, gabs_base, roh_g_cc, window):
    # Extracting Reading and time for drift correction
    reading = d.iloc[:, 1]
    s = d.iloc[:, 0]
    time = d.iloc[:, 6]
    theta = d.iloc[:, 3]

    # Coordinates of the first and last points
    x1, y1 = time.iloc[0], reading.iloc[0]
    x2, y2 = time.iloc[-1], reading.iloc[-1]

    # Calculate the distance from each point to the horizontal line
    dh = np.abs(reading - y1)

    # Calculate the distance from each point to the line connecting the first and last points
    dp = np.abs((y2 - y1) * time - (x2 - x1) * reading + x2 * y1 - y2 * x1) / np.sqrt((y2 - y1) ** 2 + (x2 - x1) ** 2)

    drift = np.abs(dh - dp)
    drift[0] = 0
    relative_reading = []

    drift_correction = reading - drift
    for i in range(len(drift_correction)):
        relative_reading.append(drift_correction[i] - drift_correction[0])

    abs_reading = [x + float(gabs_base) for x in relative_reading]

    # Theoretical readings
    theoretical_reading = []
    for i in range(len(theta)):
        theoretical_reading.append(
            978032.7 * (1 + 0.0053024 * ((np.sin(theta[i] * (np.pi / 180))) ** 2) - 0.0000058 * (
                    np.sin(2 * theta[i] * (np.pi / 180))) ** 2))
    anomaly = [abs_reading[i] - theoretical_reading[i] for i in range(len(theoretical_reading))]

    # Free-air correction
    fac = [0.3086 * ele for ele in d.iloc[:, 2]]

    fa_anomaly = [anomaly[i] + fac[i] for i in range(len(fac))]

    # Bouguer correction
    bg_corr = [0.04192 * float(roh_g_cc) * ele for ele in d.iloc[:, 2]]

    bg_anomaly = [fa_anomaly[i] - bg_corr[i] for i in range(len(bg_corr))]

    fig, ax = plt.subplots()
    ax.plot(s, bg_anomaly)
    ax.set_xlabel('Station Coordinates')
    ax.set_ylabel('Bouguer Anomaly')
    ax.set_title('Bouguer Anomaly')
    canvas = FigureCanvasTkAgg(fig, master=window)
    canvas.draw()
    canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)


def open_array_input_window():
    array_input = tk.Toplevel(root)
    array_input.title("Input Array")

    array_label = tk.Label(array_input, text="Enter the array elements separated by space:")
    array_label.pack()

    array_entry = tk.Entry(array_input)
    array_entry.pack()

    submit_button = tk.Button(array_input, text="Submit", command=lambda: get_array_input(array_entry.get()))
    submit_button.pack()


def get_array_input(array_elements):
    try:
        global input_array
        input_array = [int(elem) for elem in array_elements.split()]
        messagebox.showinfo("Success", "Array input successful!")
    except ValueError:
        messagebox.showerror("Error", "Please enter valid numeric values!")


root = tk.Tk()
root.title("Anomaly Calculator")
root.geometry("300x300")

browse_button = tk.Button(root, text="Browse File", command=browse_file)
browse_button.pack(pady=10)

gabs_base_label = tk.Label(root, text="Enter g_abs_base:")
gabs_base_label.pack()
gabs_base = tk.Entry(root)
gabs_base.pack()

roh_g_label = tk.Label(root, text="Enter roh_g (g/cc):")
roh_g_label.pack()
roh_g_entry = tk.Entry(root)
roh_g_entry.pack()

anomaly1_button = tk.Button(root, text="Theoretical Anomaly",
                            command=lambda: plot_in_new_window(theoretical_anomaly, d, gabs_base.get()))
anomaly1_button.pack(pady=5)

anomaly2_button = tk.Button(root, text="Free Air Anomaly",
                            command=lambda: plot_in_new_window(faa, d, gabs_base.get()))
anomaly2_button.pack(pady=5)

anomaly3_button = tk.Button(root, text="Bouguer Anomaly",
                            command=lambda: plot_in_new_window(ba, d, gabs_base.get(), roh_g_entry.get()))
anomaly3_button.pack(pady=5)

root.mainloop()


C:\Users\yriti\AppData\Local\Temp\ipykernel_9076\1469119173.py:8: DeprecationWarning: Please use `QhullError` from the `scipy.spatial` namespace, the `scipy.spatial.qhull` namespace is deprecated.
  from scipy.spatial.qhull import QhullError
